In [10]:
with open('openweather_key.txt', 'r') as fs:
    OPENWEATHER_KEY = fs.read().strip()
    
# AirNow.gov doesn't not give accurate readings since they are 24-hour aggregations
with open('airnow_key.txt', 'r') as fs:
    AIRNOW_KEY = fs.read().strip()

In [24]:
from datetime import datetime
from typing import Any, Dict, List, Optional

import requests

In [59]:
def get_air_quality(zipcode: str, date: str) -> Optional[List[Dict[str, Any]]]:
    base_url = "http://www.airnowapi.org/aq/forecast/zipCode/"
    params = {
        "format": "application/json",
        "zipCode": zipcode,
        "date": date,
        "distance": "50",
        "API_KEY": AIRNOW_KEY
    }
    resp = requests.get(base_url, params)
    return None if resp.status_code != 200 else resp.json()

In [60]:
def get_weather(zipcode: str) -> Dict[str, Any]:
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "zip": zipcode,
        "appid": OPENWEATHER_KEY
    }
    resp = requests.get(base_url, params)
    return None if resp.status_code != 200 else resp.json()

In [61]:
def kelvins_to_fahrenheit(k: float) -> float:
    return (k - 273.15) * (9/5) + 32

In [62]:
today: str = datetime.now().date().isoformat()

lodging = {
    "Home": "95124",
    "Bend": "97702",
    "Hood River": "97031",
    "Cannon Beach": "97138",
    "Klamath Falls": "97601",
}

In [66]:
for place, zipcode in lodging.items():
    msg = f"{place}..."
    
    weather = get_weather(zipcode)
    if weather:
        description = "+".join([d['main'] for d in weather['weather']])
        temperature = kelvins_to_fahrenheit(weather['main']['temp'])
        feels_like = kelvins_to_fahrenheit(weather['main']['feels_like'])
        msg += f"\n\t{description}\n\tTemp: {temperature:.2f}F (Feels like {feels_like:.2f}F)"
    else:
        msg += "\n\tCould not get weather data."
    
    air = get_air_quality(zipcode, today)
    if air:
        air_now = air[0]
        msg += f"\n\tAQI: {air_now['AQI']} ({air_now['Category']['Name']})"
    else:
        msg += "\n\tCould not get air quality data."
    
    print(msg + "\n")

Home...
	Smoke+Haze
	Temp: 67.46F (Feels like 67.42F)
	AQI: 102 (Unhealthy for Sensitive Groups)

Bend...
	Clear
	Temp: 84.52F (Feels like 74.07F)
	AQI: -1 (Moderate)

Hood River...
	Clear
	Temp: 86.43F (Feels like 80.40F)
	AQI: -1 (Moderate)

Cannon Beach...
	Mist
	Temp: 62.17F (Feels like 59.02F)
	Could not get air quality data.

Klamath Falls...
	Smoke
	Temp: 86.38F (Feels like 79.47F)
	Could not get air quality data.

